<a href="https://colab.research.google.com/github/rooksandkings/me8873sta/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 1:<br>
a) <center> 
    $Z=X_2-X_1$<br><br>
    \begin{align}
    var[\bar{\mu_z}] &= E\big[\bar{\mu_z}-E[\bar{\mu_z}]\big]^2 \\
    &= E\big[\frac{1}{N}\Sigma_{i=1}^N Z_i - E[\frac{1}{N}\Sigma_{i=1}^N Z_i]\big]^2\\
    &= \frac{1}{N^2}E\big[\Sigma_{i=1}^N Z_i - N*E[Z]\big]^2 \\
    &= \frac{1}{N^2}E\big\{\big[Z_1-E[Z]\big] + \big[Z_2-E[Z]\big] + ⋯ \big \}^2 \\
    &= \frac{1}{N^2}\big\{E\big[Z_1 - E[Z_1]\big]^2 + E\big[Z_2 - E[Z_2]\big]^2 + ⋯\big\}\\
    &=\frac{1}{N^2}\big\{N*var(Z)\big\}\\
    &=\frac{1}{N}var(Z)\\
    &=\frac{1}{N}var(X_2-X_1)\\
    &= \frac{1}{N}\big[var(X_2) - var(X_1)\big]
    \end{align}<br><br>
    \begin{align}
    \sigma_{\mu_z} &= \sqrt{var(\mu_z)}\\
    &= \frac{1}{\sqrt{N}}\big[\sigma_{X_2} - \sigma_{X_1}\big]
    \end{align}
   </center>

<center> 
    \begin{align}
    var[\bar{\mu_z}] &= E\big[\bar{\mu_z}-E[\bar{\mu_z}]\big]^2 \\
    &= \bar{\sigma_z} \\
    &= \frac{1}{\sqrt{N}}\sigma_z \\
    &= p(\bar{z}) \\
    &= p(\bar{z}) \\
    &= \frac{1}{\sqrt{2\pi}\sigma_\bar{z}}e^{-\frac{1}{2}}(\frac{\bar{z}-z}{\sigma_\bar{z}}) 
    \end{align}<br>
   </center>

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt